In [130]:
import seaborn as sns
sns.set(rc={'axes.facecolor': 'dimgrey', 'grid.color': 'lightgrey'})
import pandas as pd
import torch
import copy
import matplotlib.pyplot as plt
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
from pygsp import graphs
print('done')

done


In [131]:
%run utils.ipynb
%run model.ipynb

In [132]:
# class 1 is illicit, 2 is licit, unknown is unknown
labels = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edges = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
nodes = pd.read_csv("data/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)
print('done')

done


In [133]:
indexes = labels.index[labels["class"] != "unknown"].tolist()
labels_new = labels[labels["class"] != "unknown"]
nodes_new =  nodes.take(indexes)
edges_new = edges.loc[edges["txId1"].isin(labels_new["txId"])].loc[edges["txId2"].isin(labels_new["txId"])]
nodes_new.shape, labels_new.shape, edges_new.shape

((46564, 167), (46564, 2), (36624, 2))

In [134]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('done')

done


In [141]:
# define the training and test set nodes
labeled_nodes = labels.index[labels[labels["class"] != "unknown"] == True].tolist()
print(labeled_nodes)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [142]:
data, graph_info = time_step_split(nodes_new, edges_new, labels_new, device,  train_lt = 49, val_lt = 49, test_lt = 49)
data = data['train']
print(data[0])

Data(x=[2147, 165], edge_index=[2, 1924], y=[2147])


In [108]:
from torch_geometric.utils import to_dense_adj

def isConnected(edge_index):
    """
    Computes whether a graph is connected or not, based on its edge index
    :return:
    """
    adj = to_dense_adj(data['train'][0].edge_index).squeeze(0)
    g = graphs.Graph(adj)
    return g.is_connected()

In [109]:
# Define your Node2Vec model here
def defineNode2VecModel(edge_index):
    n2v = Node2Vec(edge_index=edge_index, embedding_dim=3, walk_length=20, context_size=10,
                   walks_per_node=10, num_negative_samples=1,
                   p=1, q=1, sparse=False)
    return n2v

n2v = defineNode2VecModel(data[0].edge_index)

In [110]:
#define the training function for the Node2Vec model here
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)
#########################################################################

#########################################################################
#define the test function for the Node2Vec model here
@torch.no_grad()
def test(model):
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                   z[data.test_mask], data.y[data.test_mask],
                   max_iter=150)
    return acc
#########################################################################

In [111]:
#########################################################################
# train your Node2Vec model
train_epochs = 200
# Define a loader here
loader = n2v.loader(batch_size=128, shuffle=True)
# define an optimizer here
optimizer = torch.optim.Adam(list(n2v.parameters()), lr=0.01)

for epoch in range(train_epochs):
    loss = train(n2v, loader, optimizer)
    acc = test(n2v)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
#########################################################################

TypeError: test() missing 1 required positional argument: 'model'

In [ ]:
#########################################################################
# produce embedding using the trained model
n2v.eval()
emb = n2v(torch.arange(data.num_nodes))
# visualize the features here
emb = emb.detach().numpy()
z = TSNE(n_components=2).fit_transform(emb)
y = data.y.cpu().numpy()
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(emb)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for i in range(cora_dataset.num_classes):
    plt.scatter(z[y == i,0], z[y == i,1], s=20)

plt.axis('off')
plt.show()
#########################################################################